# Пример ансамбля bagging

In [1]:
import sys

from tensorflow.keras import datasets, layers, models, losses
import numpy as np
from PIL import Image
import tensorflow as tf
import torchvision as tv
import matplotlib.pyplot as plt
from livelossplot import PlotLossesKeras
import tensorflow_addons as tfa

sys.path.append('../../')

from core.datasets import get_ds
from core.make_answer import make_ans_file
from core.ensembles import get_bagging_pred, print_bagging_ensemble_statistic

C:\Users\Salam\AppData\Roaming\Python\Python310\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
transformer = tv.transforms.Compose([
    tv.transforms.ColorJitter(brightness=.2, hue=0.2, contrast=0.5),
    tv.transforms.RandomAffine(degrees=(-10, 10), translate=(0, 0.1), scale=(0.85, 1)),
    tv.transforms.RandomPerspective(distortion_scale=0.2, p=0.7),
])

In [3]:
train_ds, val_ds, test_ds = get_ds('repaired_data_train', 'repaired_data_test', transform=transformer, one_hot=True, back=2)
val_ds_x = np.array([i[0] for i in val_ds.unbatch().as_numpy_iterator()])
val_ds_y = np.array([i[1] for i in val_ds.unbatch().as_numpy_iterator()])
val_ds_y = np.argmax(val_ds_y, axis=1)

In [4]:
model_1 = tf.keras.models.load_model(f'../checkpoints/model_g_1_categorical_accuracy.h5')
model_3 = tf.keras.models.load_model(f'../checkpoints/model_g_3_categorical_accuracy.h5')
model_4 = tf.keras.models.load_model(f'../checkpoints/model_s_smote_categorical_accuracy.h5')
model_c = tf.keras.models.load_model(f'../checkpoints/model_C_EN_1_categorical_accuracy.h5')
model_8 = tf.keras.models.load_model(f'../checkpoints/model_g_8_categorical_accuracy.h5')

models = [model_1, model_4, model_8]
names = ["model G1", "model G2", "model G8"]
# Учитывается при сохранение тестовых предиктов.
ensemble_model_name = "wunderwafel_b_test1"

было замечено что повышение итогового accuracy на пару тысячных может на кегли может увеличить результат на пару сотых. Так что не стоит делать выводы только по accuracy. 

In [9]:
print_bagging_ensemble_statistic(models=models, names=names, x_val=val_ds_x, y_val=val_ds_y)

ACCURACY SCORE
model G1: 0.9754
model G2: 0.9891
model G8: 0.984

ensemble bagging: 0.9934


делаем тестовый предикт и сохраняем:

In [5]:
import pandas as pd
wunderwafel_test = get_bagging_pred(models=models, x=test_ds)

In [6]:
ans = pd.DataFrame({'Id': np.arange(wunderwafel_test.shape[0]), 'Category': wunderwafel_test})
ans.to_csv(f"../answers/{ensemble_model_name}.csv", index=False)